In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.1. Data Cleaning & Imputation/Output')
cur_date = "041923"

library(readxl)
library(tidyverse)
library(reshape2)

# reading in files
mRNA_df = data.frame(read_excel("Input/mRNA_Data_041423.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
head(mRNA_df)

,Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,ALDH1A3,C,F,1,24,-6.6824749,4.480105
2,2,M_2,ALDH1A3,C,F,1,24,0.2560009,4.870661
3,3,M_3,ALDH1A3,C,F,1,24,0.4318636,4.879307
4,4,F_4,ALDH1A3,C,F,1,24,0.4193253,4.878692
5,6,M_6,ALDH1A3,C,F,1,24,-0.1550695,4.850246
6,1,F_1,ALDH1A3,C,F,1,72,-0.2119125,4.847400


In [3]:
# identifying the number of replicates for each sample condition
count_mRNA_df = mRNA_df %>%
    group_by(mRNA, Subject_ID, Condensate, Burn_Condition, Concentration, Time_Point) %>%
    mutate(Duplicate_Count = n()) %>% 
    arrange(-Duplicate_Count, mRNA, Subject_No, Condensate, Burn_Condition, Concentration, Time_Point) 

head(count_mRNA_df)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2,Duplicate_Count
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
6,M_6,HMOX1,PBS,PBS,NA,24,0.70848207,4.892803,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.36651641,4.876100,6
6,M_6,HMOX1,PBS,PBS,NA,24,-0.09090465,4.853452,6
6,M_6,HMOX1,PBS,PBS,NA,24,-0.37654696,4.839126,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.72283828,4.893500,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.37135970,4.876338,6


In [4]:
replicate_count_df = count_mRNA_df %>%
    group_by(Duplicate_Count) %>%
    count() %>%
    arrange(-Duplicate_Count) 

replicate_count_df

Duplicate_Count,n
<int>,<int>
6,6
5,80
4,328
3,96
2,1340
1,2361


In [5]:
# creating a new column for replicate number
replicate_vector = c()
for (i in 1:length(replicate_count_df$n)){
    # calculating number of times to repeat the replicate number from 1 to the count
    replicate_count = replicate_count_df$n[i]/replicate_count_df$Duplicate_Count[i]
    replicate_iteration = rep(1:replicate_count_df$Duplicate_Count[i], times = replicate_count)
    
    # saving iteration
    replicate_vector = c(replicate_vector, replicate_iteration)
    
}

# adding to df 
count_mRNA_df$Replicate = replicate_vector
mRNA_df = count_mRNA_df %>%
    select(-Duplicate_Count)

head(mRNA_df)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2,Replicate
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
6,M_6,HMOX1,PBS,PBS,NA,24,0.70848207,4.892803,1
6,M_6,HMOX1,PBS,PBS,NA,24,0.36651641,4.876100,2
6,M_6,HMOX1,PBS,PBS,NA,24,-0.09090465,4.853452,3
6,M_6,HMOX1,PBS,PBS,NA,24,-0.37654696,4.839126,4
6,M_6,HMOX1,PBS,PBS,NA,24,0.72283828,4.893500,5
6,M_6,HMOX1,PBS,PBS,NA,24,0.37135970,4.876338,6


Removing variables and ids with less than 25% of data. However, ids will be removed after casting the data to a wide format and back to long.

In [6]:
# only keeping variables with at least 25% of data 
variable_presence_df = mRNA_df %>% 
    # creating 2 count cols: 1st for non-missing data, 2nd for all data
    mutate(data_count = ifelse(is.na(ddCT_pslog2), 0, 1), all_count = 1) %>%
    group_by(mRNA, Condensate, Burn_Condition, Concentration, Time_Point) %>%
    summarize(Variable_Presence_Percent = sum(data_count)/ sum(all_count) * 100) %>%
    arrange(Variable_Presence_Percent)

head(variable_presence_df)

`summarise()` has grouped output by 'mRNA', 'Condensate', 'Burn_Condition',
'Concentration'. You can override using the `.groups` argument.


mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Variable_Presence_Percent
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
ALDH1A3,C,F,1,24,100
ALDH1A3,C,F,1,72,100
ALDH1A3,C,F,25,4,100
ALDH1A3,C,F,25,24,100
ALDH1A3,C,F,25,72,100
ALDH1A3,C,F,5,24,100


In [8]:
# pivoting longer but first creating sample ids
missing_data_mRNA_df = mRNA_df %>%
    # creating a column that denotes the biomarker
    #mutate(Biomarker = "mRNA") %>%
    # creating a column for the sample number to deal with replicates
    #mutate(Sample_No = 1:dim(mRNA_df)[1]) %>%
    # creating sample ids
    unite("Sample_ID", c(colnames(mRNA_df)[c(4:7,10)])) %>%
    select(-c("Subject_No", "ddCT")) %>%
    pivot_wider(names_from = "Subject_ID", values_from = "ddCT_pslog2") %>%
    pivot_longer(cols = 3:8, names_to = "Subject_ID", values_to = "ddCT_pslog2") %>%
    separate(Sample_ID, c("Condensate", "Burn_Condition", "Concentration", "Time_Point", "Replicate"))

head(missing_data_mRNA_df)

mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Replicate,Subject_ID,ddCT_pslog2
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
HMOX1,PBS,PBS,NA,24,1,M_6,4.892803
HMOX1,PBS,PBS,NA,24,1,F_5,NA
HMOX1,PBS,PBS,NA,24,1,F_1,NA
HMOX1,PBS,PBS,NA,24,1,M_2,NA
HMOX1,PBS,PBS,NA,24,1,M_3,NA
HMOX1,PBS,PBS,NA,24,1,F_4,NA


In [21]:
# only keeping ids with at least 25% of data 
subject_presence_df = missing_data_mRNA_df %>% 
    # creating 2 count cols: 1st for non-missing data, 2nd for all data
    mutate(data_count = ifelse(is.na(ddCT_pslog2), 0, 1), all_count = 1) %>%
    group_by(mRNA, Condensate, Burn_Condition, Concentration, Time_Point, Subject_ID) %>%
    summarize(Subject_Presence_Percent = sum(data_count)/ sum(all_count) * 100) %>%
    arrange(Subject_Presence_Percent)

head(subject_presence_df) 

# getting the proteins in the entire set that will be removed
removed_subjects_df = subject_presence_df %>%
    filter(Subject_Presence_Percent < 50)

# getting number of sample conditions to be removed
dim(removed_subjects_df)[1]

`summarise()` has grouped output by 'mRNA', 'Condensate', 'Burn_Condition',
'Concentration', 'Time_Point'. You can override using the `.groups` argument.


mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Subject_ID,Subject_Presence_Percent
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
ALDH1A3,C,F,1,24,F_5,0
ALDH1A3,C,F,25,4,F_5,0
ALDH1A3,C,S,1,24,F_1,0
ALDH1A3,P,F,5,24,M_6,0
ALDH3A1,C,F,1,24,M_6,0
ALDH3A1,C,F,1,72,M_6,0


[1] 420

Therefore, 420 sample conditions will be removed. 

In [23]:
# removing those 420 sample conditions
filtered_mRNA_df = anti_join(missing_data_mRNA_df, removed_subjects_df)

head(filtered_mRNA_df)

Joining, by = c("mRNA", "Condensate", "Burn_Condition", "Concentration",
"Time_Point", "Subject_ID")


mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Replicate,Subject_ID,ddCT_pslog2
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
HMOX1,PBS,PBS,NA,24,1,M_6,4.892803
HMOX1,PBS,PBS,NA,24,2,M_6,4.876100
HMOX1,PBS,PBS,NA,24,3,M_6,4.853452
HMOX1,PBS,PBS,NA,24,4,M_6,4.839126
HMOX1,PBS,PBS,NA,24,5,M_6,4.893500
HMOX1,PBS,PBS,NA,24,6,M_6,4.876338


In [25]:
filtered_mRNA_df %>%
    # creating a column that denotes the biomarker
    #mutate(Biomarker = "mRNA") %>%
    # creating a column for the sample number to deal with replicates
    #mutate(Sample_No = 1:dim(mRNA_df)[1]) %>%
    # creating sample ids
    unite("Sample_ID", c(colnames(mRNA_df)[c(4:7,10)])) %>%
    #select(-c("Subject_No", "ddCT")) %>%
    pivot_wider(names_from = "Subject_ID", values_from = "ddCT_pslog2")

mRNA,Sample_ID,M_6,F_5,F_1,M_2,M_3,F_4
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
HMOX1,PBS_PBS_NA_24_1,4.892803,NA,NA,NA,NA,NA
HMOX1,PBS_PBS_NA_24_2,4.876100,NA,NA,NA,NA,NA
HMOX1,PBS_PBS_NA_24_3,4.853452,NA,NA,NA,NA,NA
HMOX1,PBS_PBS_NA_24_4,4.839126,NA,NA,NA,NA,NA
HMOX1,PBS_PBS_NA_24_5,4.893500,NA,NA,NA,NA,NA
HMOX1,PBS_PBS_NA_24_6,4.876338,NA,NA,NA,NA,NA
ALDH3A1,PBS_PBS_NA_4_1,4.831224,4.813108,4.923408,4.827360,4.856214,4.869645
ALDH3A1,PBS_PBS_NA_4_2,4.824358,4.860738,4.866126,4.858750,4.836145,4.833120
ALDH3A1,PBS_PBS_NA_4_3,4.917834,4.910775,4.870335,4.904902,4.842222,4.884000


# QRILC Imputation